In [ ]:
# Reference for the below code is: https://haystack.deepset.ai/tutorials/03_scalable_qa_system
%%bash
pip install --upgrade pip
pip install farm-haystack[colab,preprocessing,elasticsearch,inference,metrics]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.2/150.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 86.7 MB/s e

In [ ]:
%%bash
wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg
sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

ModuleNotFoundError: No module named 'haystack'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
doc_dir = '/content/drive/My Drive/Colab Notebooks/NLP2/reference'

In [ ]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

# Initialize the TextConverter and PreProcessor with your desired configurations
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_header_footer=True,
    split_by="word",
    split_length=200,
    split_overlap=0,
    split_respect_sentence_boundary=False,
    clean_empty_lines=False,
    clean_whitespace=False,
)

# Create an indexing pipeline
indexing_pipeline = Pipeline()

# Add the TextConverter and PreProcessor to the pipeline
indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
files_to_index = [os.path.join(doc_dir, f) for f in os.listdir(doc_dir) if f.endswith('.txt')]
indexing_pipeline.run_batch(file_paths=files_to_index)

Preprocessing: 100%|██████████| 528/528 [00:14<00:00, 35.71docs/s]


{'documents': [<Document: {'content': "25 Great Things About SCS | SCS25 - Carnegie Mellon University School of Computer Science\nSkip to main content\nLegal\nEvents\nRegister\n25 Things\nHistory\nVideo\n25 Great Things About SCS\nWhat’s so great about computer science at Carnegie Mellon?We're glad you asked! Here are 25 great ideas from CMU computer scientists to think about as we celebrate the birthday of the School of Computer Science.1. Artificial intelligence, 1955-56\xa0Can you write a working computer program without a computer? Herb Simon (H’90), at left, Allen Newell (IA’57), at right, and Cliff Shaw did. The team created the first artificial intelligence program, Logic Theorist, which could solve logic puzzles in the same way that a human might solve them. Newell demonstrated that it worked by writing the instructions on 3-by-5 index cards that were manipulated on the kitchen table by Newell, his wife, and a group of Carnegie Tech grad students.\n2. Multi-core processors, 197

In [ ]:
from haystack.nodes import BM25Retriever
retriever = BM25Retriever(document_store=document_store)

In [ ]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
from haystack import Pipeline

querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Path to the question file, adjust the path according to your setup
questions_file_path = '/content/drive/My Drive/Colab Notebooks/NLP2/FAQ_style_QA/Q_Train/NLP2_Q_traning_all.txt'


Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
answers_file_path = '/content/drive/My Drive/Colab Notebooks/NLP2/model_answer.txt'

# Open and read the questions file
with open(questions_file_path, 'r') as file:
    questions = file.readlines()

# Process each question
with open(answers_file_path, 'w') as answers_file:
    # Process each question
    for question in questions:
        question = question.strip()  # Remove any leading/trailing whitespace
        if question:  # Check if the question is not empty
            prediction = querying_pipeline.run(
                query=question,
                params={"Retriever": {"top_k": 5}, "Reader": {"top_k": 1}}
            )

            # Check if there are any answers returned
            if prediction['answers']:
                # Directly access the 'answer' attribute of the first Answer object
                answer_text = prediction['answers'][0].answer
                answer_text = answer_text.replace('\n', ' ')
                con_text = prediction['answers'][0].context
                print(f"Answer: {answer_text}")
                # Write the answer to the answers file, one answer per line
                answers_file.write(f"{answer_text}\n")
            else:
                # If no answer was found, write a placeholder or a notification
                answers_file.write(" \n")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Inferencing Samples: 100%|██████████| 1/1 [00:13<00:00, 13.05s/ Batches]


Answer: August 28


Inferencing Samples: 100%|██████████| 1/1 [00:11<00:00, 11.87s/ Batches]


Answer: September 1


...

Answer: Andrew Carnegie


Inferencing Samples: 100%|██████████| 1/1 [00:09<00:00,  9.35s/ Batches]


Answer: Scottie Dog


Inferencing Samples: 100%|██████████| 1/1 [00:07<00:00,  7.20s/ Batches]

Answer: students and alumni


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Path to the question file, adjust the path according to your setup
sample_file_path = '/content/drive/My Drive/Colab Notebooks/NLP2/sample_questions.txt'

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
answers_file_path = '/content/drive/My Drive/Colab Notebooks/NLP2/roberta_sample_model_answer.txt'

# Open and read the questions file
with open(sample_file_path, 'r') as file:
    questions = file.readlines()

# Process each question
with open(answers_file_path, 'w') as answers_file:
    # Process each question
    for question in questions:
        question = question.strip()  # Remove any leading/trailing whitespace
        if question:  # Check if the question is not empty
            prediction = querying_pipeline.run(
                query=question,
                params={"Retriever": {"top_k": 5}, "Reader": {"top_k": 1}}
            )

            # Check if there are any answers returned
            if prediction['answers']:
                # Directly access the 'answer' attribute of the first Answer object
                answer_text = prediction['answers'][0].answer
                # con_text = prediction['answers'][0].context
                answer_text = answer_text.replace('\n', ' ')
                print(f"Answer: {answer_text}")
                # Write the answer to the answers file, one answer per line
                answers_file.write(f"{answer_text}\n")
            else:
                # If no answer was found, write a placeholder or a notification
                answers_file.write(" \n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Inferencing Samples: 100%|██████████| 1/1 [00:12<00:00, 12.51s/ Batches]


Answer: Buggy


Inferencing Samples: 100%|██████████| 1/1 [00:13<00:00, 13.45s/ Batches]


Answer: four semesters


...

Answer: Joan Axelson


Inferencing Samples: 100%|██████████| 1/1 [00:07<00:00,  7.03s/ Batches]


Answer: Philadelphia


Inferencing Samples: 100%|██████████| 1/1 [00:09<00:00,  9.76s/ Batches]

Answer: 1975
